# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [14]:
# Load the libraries as required.
%reload_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import dask.dataframe as dd
import pandas as pd
import numpy as np

from glob import glob
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [15]:
import pandas as pd


In [17]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [18]:
X = fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi','temp', 'rh', 'wind', 'rain']]
Y = fires_dt['area']
#X0_train, X0_test, Y0_train, Y0_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [19]:
X

coord_x  coord_y month  day  ffmc    dmc     dc   isi  temp  rh  wind  \
0          7        5   mar  fri  86.2   26.2   94.3   5.1   8.2  51   6.7   
1          7        4   oct  tue  90.6   35.4  669.1   6.7  18.0  33   0.9   
2          7        4   oct  sat  90.6   43.7  686.9   6.7  14.6  33   1.3   
3          8        6   mar  fri  91.7   33.3   77.5   9.0   8.3  97   4.0   
4          8        6   mar  sun  89.3   51.3  102.2   9.6  11.4  99   1.8   
..       ...      ...   ...  ...   ...    ...    ...   ...   ...  ..   ...   
512        4        3   aug  sun  81.6   56.7  665.6   1.9  27.8  32   2.7   
513        2        4   aug  sun  81.6   56.7  665.6   1.9  21.9  71   5.8   
514        7        4   aug  sun  81.6   56.7  665.6   1.9  21.2  70   6.7   
515        1        4   aug  sat  94.4  146.0  614.7  11.3  25.6  42   4.0   
516        6        3   nov  tue  79.5    3.0  106.7   1.1  11.8  31   4.5   

     rain  
0     0.0  
1     0.0  
2     0.0  
3     0.2  
4     0.0  
..    ...  
512   0.0  
513   0.0  
514   0.0  
515   0.0  
516   0.0  

[517 rows x 12 columns]

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [20]:
# Define feature columns
numeric_features = ['ffmc'	, 'dmc'	 , 'dc' , 'isi'	 , 'temp'	 , 'rh' , 	'wind'	, 'rain']
categorical_features = ['month' , 'day']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create ColumnTransformer
preproc1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Do not fit the transformer here, fit them in the pipeline
# Fit and transform data
#X_train_transformed = preprocessor.fit_transform(X_train)
#X_test_transformed = preprocessor.transform(X_test)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [21]:
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer

'''
Non-linear transformation
1. PolynomialFeatures:	Captures non-linear interactions (e.g., temp^2, temp * wind)
2. log of humidity: Spreads out low humidity values 
Rest are the same as preproc1

'''

# Define features
HR_features = ['temperature', 'humidity']
numeric_features = ['ffmc'	, 'dmc'	 , 'dc' , 'isi'	 , 'temp'	 , 'rh' , 	'wind'	, 'rain']
# Also keep the original variables
# The non-linear transformations are not linearly related so I can use both in a linear regression
categorical_features = ['month' , 'day']

#Define the linear transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

#non-linear transformers
poly_transformer = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
log_transformer = FunctionTransformer(np.log10, validate=True)

# Define preprocessor
preproc2 = ColumnTransformer([
   ('num', numeric_transformer, numeric_features),  # Scale temperature & humidity
    ('poly', poly_transformer, ['temp', 'wind']),  # Polynomial features
    ('log_rh', log_transformer, ['rh']),  # Log transform humidity
    ('cat', categorical_transformer, categorical_features)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [22]:
# Pipeline A = preproc1 + baseline

# Use knn as the baseline model

pipe_A = Pipeline([
        ('preprocessing', preproc1),
        ('regressor', KNeighborsRegressor())  # Linear Regression model
    ])

In [23]:
# Pipeline B = preproc2 + baseline

pipe_B = Pipeline([
        ('preprocessing', preproc2),
        ('regressor', KNeighborsRegressor())  # Linear Regression model
    ])

In [24]:
# Pipeline C = preproc1 + advanced model
pipe_C = Pipeline([
    ('preprocessing', preproc1),  # ColumnTransformer step
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))  # Regression model
])


In [25]:
# Pipeline D = preproc2 + advanced model

pipe_D = Pipeline([
    ('preprocessing', preproc2),  # ColumnTransformer step
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))  # Regression model
])      

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [27]:
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define hyperparameter grid for tuning
param_grid_knn = {
    'regressor__n_neighbors': [3, 5, 7, 9],  # Try different values of k
    'regressor__weights': ['uniform', 'distance'],  # Weighting method
    'regressor__metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metric
}

param_grid_RF = [
    {
        'regressor__n_estimators': [50, 100, 200],  # Number of trees
        'regressor__max_depth': [None, 5, 10],  # Tree depth
    }]

In [28]:
# Run Grid Search CV
grid_search_A = GridSearchCV(pipe_A, param_grid_knn, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_A.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__metric': ['euclidean', 'manhattan',
                                               'minkowski'],
                         'regressor__n_neighbors': [3, 5, 7, 9],
                         'regressor__weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

In [29]:
# Run Grid Search CV
grid_search_B = GridSearchCV(pipe_B, param_grid_knn, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_B.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('poly',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         ['temp',
                                                                          'wind']),
                                                                        ('log_rh',
                                                                         FunctionTransformer(func=<ufunc 'log10'>,
                                                                                             validate=True),
                                                                         ['rh']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__metric': ['euclidean', 'manhattan',
                                               'minkowski'],
                         'regressor__n_neighbors': [3, 5, 7, 9],
                         'regressor__weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

In [30]:
# Run Grid Search CV
grid_search_C = GridSearchCV(pipe_C, param_grid_RF, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_C.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'regressor__max_depth': [None, 5, 10],
                          'regressor__n_estimators': [50, 100, 200]}],
             scoring='neg_mean_squared_error')

In [31]:
# Run Grid Search CV
grid_search_D = GridSearchCV(pipe_D, param_grid_RF, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
grid_search_D.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('poly',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         ['temp',
                                                                          'wind']),
                                                                        ('log_rh',
                                                                         FunctionTransformer(func=<ufunc 'log10'>,
                                                                                             validate=True),
                                                                         ['rh']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'regressor__max_depth': [None, 5, 10],
                          'regressor__n_estimators': [50, 100, 200]}],
             scoring='neg_mean_squared_error')

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [32]:
all_scores = []
for model in (grid_search_A , grid_search_B, grid_search_C, grid_search_D):
    best_model = model.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    all_scores.append(rmse)

In [33]:
all_scores


[108.27984851242933,
 109.37709684029947,
 107.79140148188537,
 107.88613510561306]

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [36]:
import pickle

# Save the best model as a pickle file
best_model_filename = "best_fire_model.pkl"
with open(best_model_filename, 'wb') as file:
    pickle.dump(grid_search_C, file)

print(f"Best fire prediction model saved as: {best_model_filename}")

Best fire prediction model saved as: best_fire_model.pkl


*(Answer here.)*

In [37]:
import shap
best_model = grid_search_C.best_estimator_  # Get the best model

# Ensure X_test is converted to NumPy
X_test_array = X_test.to_numpy()

# Wrap the predict function to avoid attribute errors
def pipeline_predict(X):
    return best_model.predict(pd.DataFrame(X, columns=X_test.columns))

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(pipeline_predict, shap.sample(X_train, 100).to_numpy())

# Compute SHAP values
shap_values = explainer.shap_values(X_test_array)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 104/104 [00:38<00:00,  2.67it/s]


## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

ðŸš¨ **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** ðŸš¨ for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.